In [2]:
#importer les librairies
import os
import cv2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import skimage as skimage
%matplotlib inline
from xgboost import XGBClassifier
import xgboost as xgb
from sklearn.model_selection import train_test_split
from skimage.feature import hog
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

In [3]:
#constant
image_height = 350
image_width = 350

In [4]:
# importer les donnees

def extract_image_data(src_folder):
    image_data = []
    labels = []
    
    for label in os.listdir(src_folder):
        label_folder = os.path.join(src_folder, label)
        if os.path.isdir(label_folder):
            for image_file in os.listdir(label_folder):
                image_path = os.path.join(label_folder, image_file)
                image = cv2.imread(image_path)
                image = cv2.resize(image, (image_height,image_width))  # Resize image if necessary
                image_data.append(image.flatten())  # Flatten image and add to data
                labels.append(label)  # Add corresponding label

    return np.array(image_data), np.array(labels)



In [6]:
# data, labels = extract_image_data("C:/Users/Airperssone/Downloads/data/data/val")
from images import load_images_from_folder
data, labels, folder_names, smallest_height, smallest_width, smallest_height2, smallest_width2 = load_images_from_folder(folder_path)

NameError: name 'folder_path' is not defined

In [ ]:
# transformer la data avec les descripteurs
def extract_hog_features(images):
    features = []
    for image in images:
        feature = hog(skimage.color.rgb2gray(image.reshape((image_height, image_width, 3))))  # Extract HOG features
        features.append(feature)
    return np.array(features)

In [ ]:
hog_data = extract_hog_features(data)

In [ ]:
label_encoder = LabelEncoder()

In [ ]:
labels_encoded = label_encoder.fit_transform(labels)

In [ ]:
#donner la data du descripteur pour entrainer le modele
X_train, X_test, y_train, y_test = train_test_split(hog_data, labels_encoded, test_size=0.2, random_state=42)

In [ ]:
params = {
    'objective': 'multi:softmax', 
    'num_class': len(np.unique(labels_encoded)),
    'eta': 0.1,  # Decrease learning rate [0.01-0.3] Lower values make the model more robust but require more boosting rounds.
    'max_depth': 8,  # Adjust max depth of trees [3-10] Higher values allow the model to capture more complex interactions in the data but can lead to overfitting.
    'num_boost_round': 100,  # Increase number of boosting rounds [50-500]
    'subsample': 0.75,  # Reduce subsample ratio [0.5-1] Lower values introduce more randomness and can prevent overfitting.
    'colsample_bytree': 0.85,  # Reduce column subsample ratio [0.5-1] Lower values introduce more randomness in feature selection.
    'lambda': 10,  # L2 regularization term [0-10+] Higher values add more regularization to prevent overfitting.
    'alpha': 10  # L1 regularization term [0-10+] Higher values add more regularization to prevent overfitting.
}

In [ ]:
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

In [ ]:
bst = xgb.train(params, dtrain, num_boost_round=params['num_boost_round'])

C:\Users\Airperssone\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\xgboost\core.py:160: UserWarning: [02:59:06] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "num_boost_round" } are not used.

  warnings.warn(smsg, UserWarning)


In [ ]:
#obtention des scores de précision
preds = bst.predict(dtest)
overfitting_score = bst.predict(dtrain)
accuracy = accuracy_score(y_test, preds)
overfitting_accuracy = accuracy_score(y_train, overfitting_score)
print("Accuracy:", accuracy)
print("Trainning Score (show overfitting):", overfitting_accuracy)

Accuracy: 0.07853403141361257
Trainning Score (show overfitting): 0.9986910994764397


In [ ]:
preds

array([16.,  5., 18.,  3.,  2., 12., 19.,  7., 15., 11., 13., 10.,  1.,
       14.,  7., 18., 16., 17., 10.,  6.,  2.,  9.,  9., 11.,  7., 17.,
        3.,  1., 19., 11., 16.,  0., 14.,  3.,  7., 17., 12., 11.,  3.,
       14.,  1.,  6., 10., 10.,  1., 19., 12.,  8.,  1.,  8.,  9., 11.,
        5., 16.,  3., 12.,  3., 11.,  1., 18., 14.,  8., 18., 16., 14.,
        4., 11., 14.,  9.,  9., 16., 16.,  1.,  9.,  7., 12., 17.,  9.,
       18.,  2., 13.,  1., 10.,  3.,  6.,  3., 10.,  0.,  2., 10.,  3.,
        2.,  2., 11.,  3., 14.,  3., 11.,  2., 11.,  1., 12.,  0.,  6.,
       10.,  9.,  0.,  9.,  9.,  0., 15., 10.,  1.,  2., 12., 11.,  3.,
       18., 13.,  5., 11., 16.,  7., 10.,  2.,  1., 10.,  8.,  3., 15.,
        0., 18., 19., 11.,  4.,  6., 13., 14.,  8.,  4., 16.,  6., 12.,
       12., 16., 17., 17.,  2., 11.,  7.,  0., 18., 17., 10.,  1., 18.,
        9.,  9., 18.,  9., 17.,  3., 17.,  0.,  3., 10.,  2., 19.,  8.,
       10.,  8.,  7.,  7., 19.,  8., 11., 14., 16.,  9.,  0.,  4

In [ ]:
y_test

array([ 3,  8, 14, 15,  7, 11, 17, 10,  9,  3,  4, 10,  1, 16, 14,  1, 18,
        7,  2,  8,  9, 13,  9,  9,  0, 13, 10, 17, 12,  0, 19, 12,  7,  5,
        1,  2,  5,  2,  8, 15,  0,  6, 10, 12,  1,  6,  4,  5,  1,  3,  5,
        1,  6, 18, 15, 15, 17,  7, 16,  5,  6,  5, 19,  6,  3,  1, 16, 12,
       17,  6, 18,  8,  1,  6, 19, 19, 13, 10, 15, 16, 13, 10, 11, 18, 18,
        4, 11, 10,  3, 10, 17,  5,  9, 12,  6, 14,  1, 10,  0, 19, 13, 11,
       11,  4,  5, 13, 15,  5,  5,  6,  6, 19,  2, 10,  7, 14, 10, 18,  6,
       12,  1,  9, 17, 12, 13, 16,  7,  1,  7, 11, 18, 14, 10,  5,  2, 18,
        8, 16,  9,  4, 18,  7,  7, 11, 18, 11,  2, 16,  6, 12, 14,  3, 10,
        3, 11, 18, 19,  1, 12,  5,  4,  4, 12, 10,  8,  1,  2, 16,  7,  6,
       12, 13,  7,  7,  8, 18,  0, 14,  9,  4, 18, 16,  6, 19, 16, 10,  0,
        0, 14,  6,  9], dtype=int64)

In [ ]:
#comparaison et visualisation